This example is not yet full functional (getRawAvgTables is missing)

This example illustrates the execution on a layered queueing network model.
Performance indexes now refer to processors, tasks, entries, and activities.
Indexes refer to the submodel (layer) where the processor or task acts as a server.
NaN indexes indicate that the metric is not supported by the node type.

In [1]:
from line_solver import *
GlobalConstants.setVerbose(VerboseLevel.SILENT)

In [2]:
model = LayeredNetwork('myLayeredModel')
P = np.empty(2, dtype=object)
P[0] = Processor(model, 'P1', 1, SchedStrategy.PS)
P[1] = Processor(model, 'P2', 1, SchedStrategy.PS)
T = np.empty(2, dtype=object)
T[0] = Task(model, 'T1', 10, SchedStrategy.REF).on(P[0]).setThinkTime(Exp.fitMean(100))
T[1] = Task(model, 'T2', 1, SchedStrategy.FCFS).on(P[1]).setThinkTime(Immediate())
E = np.empty(2, dtype=object)
E[0] = Entry(model, 'E1').on(T[0])
E[1] = Entry(model, 'E2').on(T[1])
A = np.empty(4, dtype=object)
A[0] = Activity(model, 'AS1', Exp.fitMean(1.6)).on(T[0]).boundTo(E[0])
A[1] = Activity(model, 'AS2', Immediate()).on(T[0]).synchCall(E[1],1)
A[2] = Activity(model, 'AS3', Exp.fitMean(5.0)).on(T[1]).boundTo(E[1])
A[3] = Activity(model, 'AS4', Exp.fitMean(1.0)).on(T[1]).repliesTo(E[1])
T[0].addPrecedence(ActivityPrecedence.Serial(A[0], A[1]))
T[1].addPrecedence(ActivityPrecedence.Serial(A[2], A[3]))

In [3]:
solver = np.array([], dtype=object)

solver = np.append(solver, SolverLQNS(model))
solver = np.append(solver, SolverLN(model))

AvgTable = np.empty(len(solver), dtype=object)
for s in range(len(solver)):
    print(f'\nSOLVER: {solver[s].getName()}')    
    print(solver[s].getAvgTable())
    
# useLQNSnaming = true
# [AvgTable[2], CallAvgTable[2]] = solver[0].getRawAvgTables()
# AvgTable[2]
# CallAvgTable[2]


SOLVER: SolverLQNS
  Node   NodeType    QLen        Util    RespT  ResidT  ArvR    Tput
0   P1  Processor     NaN  1.4206e-01      NaN     0.0   0.0     NaN
1   P2  Processor     NaN  5.3271e-01      NaN     0.0   0.0     NaN
2   T1       Task  1.1215  1.4206e-01      NaN     0.0   0.0  0.0888
3   T2       Task  0.5327  5.3271e-01      NaN     0.0   0.0  0.0888
4   E1      Entry  1.1215  1.4206e-01  12.6310     0.0   0.0  0.0888
5   E2      Entry  0.5327  5.3271e-01   6.0000     0.0   0.0  0.0888
6  AS1   Activity  0.1627  1.4206e-01   1.8327     0.0   0.0  0.0888
7  AS2   Activity  0.9587  8.8786e-10  10.7983     0.0   0.0  0.0888
8  AS3   Activity  0.4439  4.4393e-01   5.0000     0.0   0.0  0.0888
9  AS4   Activity  0.0888  8.8786e-02   1.0000     0.0   0.0  0.0888

SOLVER: SolverLN
  Node   NodeType    QLen    Util    RespT  ResidT  ArvR    Tput
0   P1  Processor     NaN  0.1421      NaN     NaN   NaN     NaN
1   P2  Processor     NaN  0.5329      NaN     NaN   NaN     NaN
2   T1  